### A2.5.3. Lowering to LLVM

> *Dialect lowering in MLIR progressively converts operations from higher-level dialects to lower-level ones, ultimately reaching the LLVM dialect for code generation.*

**Explanation:**

Lowering is the process of converting operations from one dialect to another, typically from a higher abstraction to a lower one. The final target is the **LLVM dialect**, which maps one-to-one to LLVM IR and can be exported as a `.ll` file.

**Key Components:**

- **ConversionTarget** — declares which dialects/operations are *legal* (allowed in output), *illegal* (must be converted), or *dynamically legal* (legal under certain conditions).

- **TypeConverter** — maps types between dialects (e.g., `memref<4xf32>` → `!llvm.ptr` + metadata).

- **ConversionPattern** — like a `RewritePattern` but with access to the type converter and adapted operands (already converted to target types).

**Typical Lowering Pipeline:**

```
linalg → affine → scf → cf → llvm
  ↓        ↓       ↓     ↓     ↓
tensor  memref   arith  arith  llvm
```

Each arrow represents a conversion pass. Operations are converted level by level — partial lowering is allowed (some ops stay, some convert).

**Partial vs. Full Conversion:**

- `applyPartialConversion` — converts what it can, leaves legal ops untouched.
- `applyFullConversion` — fails if any illegal op remains after conversion.

**Example:**

```mlir
// Before (scf + arith):
scf.for %i = %lb to %ub step %step {
  %sum = arith.addi %a, %b : i32
}

// After (llvm):
llvm.br ^bb1(%lb)
^bb1(%i):
  %sum = llvm.add %a, %b : i32
  %next = llvm.add %i, %step : i64
  %cond = llvm.icmp "slt" %next, %ub : i64
  llvm.cond_br %cond, ^bb1(%next), ^bb2
^bb2:
  ...
```

In [ ]:
from dataclasses import dataclass


@dataclass
class IROperation:
    dialect: str
    name: str
    operands: list[str]
    result: str

    @property
    def qualified_name(self):
        return f"{self.dialect}.{self.name}"


class ConversionTarget:
    def __init__(self):
        self.legal_dialects = set()
        self.illegal_dialects = set()

    def add_legal_dialect(self, dialect):
        self.legal_dialects.add(dialect)

    def add_illegal_dialect(self, dialect):
        self.illegal_dialects.add(dialect)

    def is_legal(self, operation):
        return operation.dialect in self.legal_dialects


LOWERING_MAP = {
    ("arith", "addi"): ("llvm", "add"),
    ("arith", "muli"): ("llvm", "mul"),
    ("arith", "subi"): ("llvm", "sub"),
    ("scf", "for"): ("llvm", "br"),
    ("func", "call"): ("llvm", "call"),
}


def lower_operation(operation):
    key = (operation.dialect, operation.name)
    if key not in LOWERING_MAP:
        return operation
    target_dialect, target_name = LOWERING_MAP[key]
    return IROperation(target_dialect, target_name, operation.operands, operation.result)


def apply_full_conversion(operations, target):
    lowered = [lower_operation(op) for op in operations]
    illegal_remaining = [
        op for op in lowered
        if not target.is_legal(op)
    ]
    return lowered, illegal_remaining


operations = [
    IROperation("arith", "addi", ["%x", "%y"], "%a"),
    IROperation("arith", "muli", ["%a", "%z"], "%b"),
    IROperation("scf", "for", ["%lb", "%ub", "%step"], "%loop"),
    IROperation("func", "call", ["%b"], "%result"),
]

target = ConversionTarget()
target.add_legal_dialect("llvm")
target.add_illegal_dialect("arith")
target.add_illegal_dialect("scf")
target.add_illegal_dialect("func")

print("Before lowering:")
for op in operations:
    print(f"  {op.result} = {op.qualified_name}({', '.join(op.operands)})")

lowered_ops, illegal = apply_full_conversion(operations, target)

print("\nAfter lowering:")
for op in lowered_ops:
    print(f"  {op.result} = {op.qualified_name}({', '.join(op.operands)})")

print(f"\nAll operations legal: {len(illegal) == 0}")

**References:**

[📘 MLIR Project. *Dialect Conversion.*](https://mlir.llvm.org/docs/DialectConversion/)

[📘 Lattner, C. et al. (2021). *MLIR: Scaling Compiler Infrastructure for Domain Specific Computation.* IEEE CGO.](https://ieeexplore.ieee.org/document/9370308)

---

[⬅️ Previous: Pattern Rewriting](./02_pattern_rewriting.ipynb)